In [2]:
def read_csv_list(my_list, my_tag):
    tag_name = my_tag
    file_name = tag_name + '_tag_link.csv'
    
    with open(file_name) as file:
        for line in file:
            read_data = line
            my_list.append(read_data[:-2])
        file.close()

my_tag = 'seoul'
seoul_links = []
read_csv_list(seoul_links, my_tag)
print(seoul_links[0])
print(len(seoul_links))

https://www.instagram.com/p/BsaIeBsBE8E/
3423


In [3]:
## 추출하고자 하는 내용은 다음과 같습니다.

## id ( 게시글 아이디 ) -> OK
## username ( 유저 아이디) -> OK
## date ( 게시 날짜 ) -> OK
## contents ( 글 내용 ) -> OK
## tags ( 태그들 ) -> OK
## image_links ( 이미지 링크들 ) -> OK
## like_num ( 좋아요 개수 ) -> OK
## comment_num ( 댓글 개수 ) -> OK

In [4]:
import requests
import re
import time
import json
from bs4 import BeautifulSoup

In [5]:
## 찾아낸 데이터를 JSON 객체로 생성하는 함수를 정의합니다.
def data2json(my_tag, id, username, date, contents, hash_tags, final_image_link, likes_num, comments_num):
    single_data = {
        "find_tag" : my_tag,
        "id" : id,
        "username" : username,
        "date" : date,
        "contents" : contents,
        "hashtags" : hash_tags,
        "imagelinks" : final_image_link,
        "likes" : likes_num,
        "comments" : comments_num
    }
    return single_data

In [7]:
json_list = []

for i in range(len(seoul_links[:300])-1):
    
    test_url = seoul_links[i]
    req = requests.get(test_url)
    html = req.text
    header = req.headers
    status = req.status_code
    soup = BeautifulSoup(html, 'html.parser')
    script_content = soup.find_all('script')

    ## 게시글 아이디를 링크로부터 가져와 찾아냅니다.
    id_p = re.compile("\/p\/(.*?)\/")
    id = id_p.findall(test_url)[0]

    ## 사용자 이름을 rel='canonical'에서 주소를 가져와 찾아냅니다.
    user_content = soup.find(rel="canonical")
    if user_content:
        username_pp = user_content.get('href')
        username_p = re.compile("instagram.com/(.*)/p")
        username = username_p.findall(str(username_pp))
        if len(username) > 0:
            username = username[0]


    ## 날짜를 header 파일에서 'Date' 부분을 가져와 찾아냅니다.
    date_p = re.compile("uploadDate\":\"(.{19})\"")
    date = date_p.findall(str(script_content))
    if len(date) > 0:
        date = date[0]
    else:
        date = 0
        

    ## 글 내용을 script 내용에서 caption 부분을 가져와 찾아냅니다.
    ## 태그 내용을 contents 에서 빼는 과정이 필요할지도 모릅니다.
    if script_content:   
        contents_p = re.compile("\"caption\":\"(.*?)\"")
        contents = contents_p.findall(str(script_content))
        
        if len(contents) > 0:
            contents = contents[0]
            contents = contents.encode('utf-8')
            contents = contents.decode('unicode_escape')
            contents = contents.encode('utf-8','ignore')
            contents = contents.decode('utf-8')
            

    ## 해쉬태그를 property='instapp:hashtags' 에서 contetn= 이후 부분을 가져와 찾아냅니다.
    meta_content = soup.find_all(property = "instapp:hashtags")
    if meta_content:
        hash_tags_p = re.compile("content=\"(.*?)\"")
        hash_tags = hash_tags_p.findall(str(meta_content))
        

    ## 이미지 링크를 display_resources 에서 가져와 찾아냅니다.
    ## 이미지가 N개일 때, 총 N+1 종류의 이미지 링크가 존재합니다.
    ## 확인해 보니 1번째와 2번째 이미지는 동일하며, 나머지는 서로 다른 이미지였습니다.
    ## 추정해 보면 1번째 이미지는 2번째 이후 이미지들을 대표하는 하나의 이미지라 할 수 있습니다.
    ## 각 이미지에 대해 서로 다른 resolution 으로 3가지의 다른 형태로 저장된 것을 볼 수 있습니다.
    ## 1번째는 640, 2번째는 750, 3번째는 원본 크기인것으로 보이며, 이 중에서 원본 크기의 링크만 저장합니다.
    image_links_p = re.compile("\"display_resources\":\[.*?\]")
    image_links = image_links_p.findall(str(script_content))

    final_image_link = []
    single_image_link_p = re.compile("\"src\":\"(.*?)\"")
    for dif_img in image_links:
        link = single_image_link_p.findall(str(dif_img))[2]
        final_image_link.append(link)
        

    ## 이미지가 1개일 때는 대표 이미지만 존재하므로, 이에 대한 예외처리가 필요하다
    if len(final_image_link) != 1:
        final_image_link = final_image_link[1:]    

    ## like_num, comment_num 을 description 에서 찾아냅니다.
    likes_num_p = re.compile("\"description\":\"(.*?)Likes")
    likes_num = likes_num_p.findall(str(script_content))
    if len(likes_num) > 0:
        likes_num = likes_num[0]
    else:
        likes_num = 0
        
    comments_num_p = re.compile("\"description\":\".*?,(.*?)Comments")
    comments_num = comments_num_p.findall(str(script_content))
    if len(comments_num) > 0:
        comments_num = comments_num[0]
    else:
        comments_num = 0
    

    single_json = data2json(my_tag, id, username, date, contents, hash_tags, final_image_link, likes_num, comments_num)
    json_list.append(single_json)
    time.sleep(0.1)
    if i % 100 == 0:
        print(i, '번째 데이터')
        print(single_json)
        print('------------------')

0 번째 데이터
{'find_tag': 'seoul', 'id': 'BsaIeBsBE8E', 'username': 'u.mizuki__1201', 'date': '2019-01-09T09:11:51', 'contents': '\u202a\u202a☺︎\u202c\n.\n.\n2019.01.09\n.\n.全部美味しかった\n.\n.\n.\n.\n.\n.\n#梅田#梅田カフェ\n#韓国#カフェ#ソウルラブ \n#チーズ#チーズハットグ#韓国料理\n#Seoul#cafe#cheese#\n#instagood #instalike #♡', 'hashtags': ['カフェ', '韓国', 'instagood', 'instalike', '梅田', 'チーズハットグ', 'seoul', 'cafe', 'cheese', '梅田カフェ', '韓国料理', 'チーズ', 'ソウルラブ', '🧀'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/a4909194f53390110bb2554d62ddc4b8/5CD40E75/t51.2885-15/e35/47585610_273424850000153_5509265993192751873_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com', 'https://scontent-icn1-1.cdninstagram.com/vp/71b18e560a86841c5ea8ac5f72b256bb/5CBEFFB6/t51.2885-15/e35/49571248_1610772119022869_7110727206641766959_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com', 'https://scontent-icn1-1.cdninstagram.com/vp/72197df100a7e643a608d3dfab9e3352/5CD0E26B/t51.2885-15/e35/47694916_1193666707447391_4948432914143032963_n.jpg?_nc_ht=sco

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: invalid escape sequence '\/'


100 번째 데이터
{'find_tag': 'seoul', 'id': 'BsaVA_xnznJ', 'username': 'dkguide', 'date': '2019-01-09T11:01:29', 'contents': '음식점: 교대점 펍피맥\n\n맛:\n음식 디자인: \n가게 분위기:  메뉴: 포테이토 핫포테이토 베이컨 피자! -“존맛탱 피자”\n- -\n #seoul #korea #강남역맛집 #강남역 #교대맛집 #서초맛집 #jmt #맛스타그램 #강남맛집 #피자 #피자맛집 #존맛탱 #펍피맥 #교대역 #서초동 #dkguide', 'hashtags': ['korea', '강남맛집', '서초맛집', '존맛탱', '강남역', '맛스타그램', '펍피맥', 'dkguide', 'jmt', '교대역', '강남역맛집', 'seoul', '피자맛집', '서초동', '교대맛집', '피자'], 'imagelinks': ['https://scontent-icn1-1.cdninstagram.com/vp/d4735b4804cd590a0dec3afd21bdaad3/5CC58B5A/t51.2885-15/e35/49803767_500065490516522_1745647490491657696_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com', 'https://scontent-icn1-1.cdninstagram.com/vp/90f1fdb0e45a2e635fed7a34e2a5679f/5CD77EF8/t51.2885-15/e35/47692300_326556731524374_4594817426198896552_n.jpg?_nc_ht=scontent-icn1-1.cdninstagram.com', 'https://scontent-icn1-1.cdninstagram.com/vp/b1a1a037d40ea15d5f059a3c1a266d20/5CD3A670/t51.2885-15/e35/49406683_137535667152098_4584054789812669939_n.jpg?_

In [8]:
len(json_list)

299

In [9]:
## 생성한 데이터를 파일 형태로 저장합니다.

def save_jsonlist_json(myjson):
    file_name = my_tag + '_rawdata.json'
    with open(file_name, "w") as file:
        for line in myjson:
            file.write(str(line) + ',\n')

save_jsonlist_json(json_list)

In [10]:
## 앞으로 해야할 일

## 1. Like 와 Comment 형태를 1,000 등의 string 에서 1000 의 int 로 변환 저장

## !. time.sleep 을 주지 않고 크롤링 하니 300번째 정도에서 104 error 가 발생하였다
##    밴인지는 정확하게 모르겠지만, 몇번 더 확인해볼 필요가 있다. 0.1 sleep 에서는 정상작동 하였다.
##    여기서 점차 sleep time 을 줄여가며 속도를 올릴 필요가 있다